In [217]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


In [165]:
#Base del SIAF diario
path = "/content/drive/My Drive/02 Siafdata/REP_INV_SIAF.xlsx" #Base actualizada de la inversión pública, se actualizada todos los días a las 10 p.m.
df = pd.read_excel(path)
df.drop_duplicates()
df["CUI"]=df["ACT_PROY"]


#Base de Presupuesto 2024
"""
La base se encuentra por CUI y SEC_EJEC,
"""
path = "/content/drive/My Drive/2024/Pptp_2024_Inversiones.xlsx" #SIAF 2024
df_2024 = pd.read_excel(path, sheet_name="GNGRGL_2024")
df_2024.drop_duplicates()
# Dividir la columna "PRODUCTO_PROYECTO" en dos columnas
df_2024['CUI'] = df_2024['PRODUCTO_PROYECTO'].str.split('.').str[0].str.strip()
df_2024['Nombre de inversión'] = df_2024['PRODUCTO_PROYECTO'].str.split('.').str[1].str.strip()


#Base REP_Inventario F12B
path = "/content/drive/My Drive/02 Siafdata/REP_INV_F12B.xlsx" #SIAF 2024
df_f12b = pd.read_excel(path)
df_f12b.drop_duplicates()
df_f12b["CUI"]=df_f12b["CODIGO_UNICO"]


In [166]:
df_ppt=df_2024.groupby("CUI")["PPTO_2024"].sum().to_frame()

In [189]:
df_ppt_reset = df_ppt.reset_index()
df_f12b['CUI'] = df_f12b['CUI'].astype(int)
df_ppt_reset['CUI'] = df_ppt_reset['CUI'].astype(int)
df_final = df_f12b.merge(df_ppt_reset, on='CUI', how='left')
df_final["Avance"]=df_final["DEVENG_ACUMULADO"]/df_final["COSTO_ACTUALIZADO"]
df_final["Monto_de"]=df_final["COSTO_ACTUALIZADO"]-df_final["DEVENG_ACUMULADO"]

In [190]:
var_key=df_final[["COSTO_ACTUALIZADO", "Avance", "Monto_de","PPTO_2024", "DES_ULT_SITUACION", "CUI", "NOMBRE_PIP"]]

In [191]:
var_key

,COSTO_ACTUALIZADO,Avance,Monto_de,PPTO_2024,DES_ULT_SITUACION,CUI,NOMBRE_PIP
0,13618688.25,0.117701,12015756.61,7076891.0,LA INVERSIÓN SE ENCUENTRA EN PROCESO DE LICITA...,2344575,MEJORAMIENTO DE LOS SERVICIOS DE EDUCACIÓN INI...
1,5893556.63,0.996666,19646.70,NaN,EL PROYECTO SE EJECUTO BAJO LA MODALIDAD DE AD...,2195593,"INSTALACION, MEJORAMIENTO Y AMPLIACION DEL SIS..."
2,14206988.33,0.998548,20628.27,NaN,LA OBRA CONTRACTUAL CONSTA DE 07 ITEMS LA CUAL...,2300025,MEJORAMIENTO DE LOS SERVICIOS DE EDUCACION INI...
3,9421363.14,0.998500,14127.61,NaN,Durante el presente la EJECUCIÓN programada de...,2322620,MEJORAMIENTO E INSTALACION DE LOS SERVICIOS DE...
4,28540430.43,0.921241,2247827.51,NaN,"OBRA CULMINADA, RECEPCIONADA, LIQUIDADA Y TRAN...",2200458,MEJORAMIENTO DE LOS SERVICIOS DEL DESEMBARCADE...
...,...,...,...,...,...,...,...
49520,4013021.20,0.000000,4013021.20,NaN,NaN,2576483,CREACION DEL SERVICIO DE AGUA POTABLE RURAL Y ...
49521,3532019.68,0.004483,3516185.68,NaN,PROYECTO EN FORMULACION DE EXPEDIENTE TECNICO ...,2596250,MEJORAMIENTO Y AMPLIACION DEL SERVICIO DE AGUA...
49522,5391129.07,0.000000,5391129.07,NaN,FECHA DE VIABILIDAD: 07/07/2023.,2601612,MEJORAMIENTO DE LOS SERVICIOS PÚBLICOS DE INTE...
49523,496231.86,0.036273,478231.86,NaN,NaN,2578706,CONSTRUCCION DE SISTEMA DE ABASTECIMIENTO DE A...


In [206]:
def obtener_texto_por_codigo(df, codigos):
    textos = []

    for codigo in codigos:
        fila = df[df['CUI'] == codigo]

        if fila.empty:
            textos.append(f"Código {codigo} no encontrado en el DataFrame.")
            continue

        avance_format = round(fila['Avance'].values[0] * 100)
        texto_formateado = (
            str(fila['CUI'].values[0]) + " " + str(fila['NOMBRE_PIP'].values[0]) + "\n"
            "Costo Actualizado: S/ " + '{:,.0f}'.format(fila['COSTO_ACTUALIZADO'].values[0]).replace(',', ' ') + "\n"
            f"Avance: {avance_format}%" + "\n"
            "Monto necesario para completar: S/ " + '{:,.0f}'.format(fila['Monto_de'].values[0]).replace(',', ' ') + "\n"
            "Tiene PIA 2024 (PL PPTO 2024): S/ " + '{:,.0f}'.format(fila['PPTO_2024'].values[0]).replace(',', ' ') + "\n"
            "Comentarios: " + str(fila['DES_ULT_SITUACION'].values[0])
        )

        textos.append(texto_formateado)

    return textos



In [209]:
codigos_input = [2191657, 2384330, 2092121]  # Reemplaza con la lista de códigos que quieras.
resultados = obtener_texto_por_codigo(var_key, codigos_input)

for texto in resultados:
    print(texto)
    print("-" * 100)  # Línea divisoria para mejor visualización entre textos.


2191657 MEJORAMIENTO DEL SERVICIO DE READAPTACION SOCIAL EN EL ESTABLECIMIENTO PENITENCIARIO DE CHINCHA
Costo Actualizado: S/ 92 640 190
Avance: 67%
Monto necesario para completar: S/ 30 148 518
Tiene PIA 2024 (PL PPTO 2024): S/ 80 000
Comentarios: INFRAESTRUCTURA: la obra se encuentra resuelta desde el 11/08/2021 (resolución del Contrato N°006-2018-INPE-OIP de fecha 22/05/2018) mediante Carta Notarial 26969-21 por la Contratista H Y HE Contratistas Generales SAC y se culminó hasta la valorización 28 de fecha 06/01/2021 encontrándose con un avance del 64.40%. adicionalmente se efectuó el Acta de constatación física e inventario de obra y posterior notificación del Laudo Arbitral producto de la resolución de contrato.
- COMPONENTE EQUIPAMIENTO (Equipamiento Electrónico): Pendiente pago de gastos administrativos en ejecución del Laudo arbitral expediente Nº 3115-487-20-PUCP por un monto de S/ 14,361.34.
- Se contrató el servicio de consultoría para la elaboración del estudio hidrogeológi

In [210]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving prueba.xlsx to prueba.xlsx
User uploaded file "prueba.xlsx" with length 15891 bytes


In [211]:
if fn.endswith('.txt'):
    # Suponiendo que los códigos están en un archivo llamado 'codigos.txt', una línea por código:
    with open(fn, 'r') as file:
        codigos_input = [int(line.strip()) for line in file.readlines()]

elif fn.endswith('.xlsx'):
    # Si el archivo es un Excel, suponemos que los códigos están en la primera columna del primer sheet.
    df_codigos = pd.read_excel(fn)
    codigos_input = df_codigos.iloc[:, 0].tolist()

else:
    print("Formato de archivo no reconocido.")


In [215]:
resultados = obtener_texto_por_codigo(var_key, codigos_input)

for texto in resultados:
    print(texto)
    print("-" * 100)  # Línea divisoria para mejor visualización entre textos.



2344575 MEJORAMIENTO DE LOS SERVICIOS DE EDUCACIÓN INICIAL ESCOLARIZADA EN LAS II.EE. N 376 - EL ROLLO, N 388 SAN ISIDRO, N 541 MIRAFLORES, N 551 SAN PEDRO, DISTRITO DE CHOROS - CUTERVO - CAJAMARCA
Costo Actualizado: S/ 13 618 688
Avance: 12%
Monto necesario para completar: S/ 12 015 757
Tiene PIA 2024 (PL PPTO 2024): S/ 7 076 891
Comentarios: LA INVERSIÓN SE ENCUENTRA EN PROCESO DE LICITACIÓN DE OBRA.  INFORMACIÓN ACTUALIZADA AL MES DE AGOSTO DEL 2023.
----------------------------------------------------------------------------------------------------
2195593 INSTALACION, MEJORAMIENTO Y AMPLIACION DEL SISTEMA DE AGUA POTABLE Y LETRINIZACION EN LAS LOCALIDADES DE UNION BAJA, MACHETILLO, LOS ANGELES, TABLONCILLO Y LAS COCAS, DISTRITO DE LA COIPA,, PROVINCIA DE SAN IGNACIO - CAJAMARCA
Costo Actualizado: S/ 5 893 557
Avance: 100%
Monto necesario para completar: S/ 19 647
Tiene PIA 2024 (PL PPTO 2024): S/ nan
Comentarios: EL PROYECTO SE EJECUTO BAJO LA MODALIDAD DE ADMINISTRACION DIRECTA, 

In [216]:
# Escribir los resultados en un archivo txt
with open('resultados.txt', 'w') as file:
    for texto in resultados:
        file.write(texto)
        file.write("\n" + "-" * 100 + "\n")

# Descargar el archivo
files.download('resultados.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>